In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [8]:
import random
from collections import defaultdict

class BlackjackMCES:
    def __init__(self):
        self.Q = defaultdict(lambda: defaultdict(float))  # 状態-行動価値関数
        self.Returns = defaultdict(lambda: defaultdict(list))  # 報酬リスト
        self.policy = {}  # 方策（状態 -> 行動）

    def initialize_policy(self):
        """初期方策をランダムに設定"""
        for player_sum in range(12, 22):  # プレイヤーの手札合計（12~21）
            for dealer_card in range(1, 11):  # ディーラーの表向きカード（1~10）
                for usable_ace in [True, False]:  # エースの有無
                    state = (player_sum, dealer_card, usable_ace)
                    self.policy[state] = random.choice(["hit", "stick"])

    def draw_card(self):
        """カードを引く（1~10のランダムな値）"""
        return random.randint(1, 10)

    def calculate_hand(self, hand):
        """手札の合計値とエースの利用可否を計算"""
        total = sum(hand)
        if 1 in hand and total + 10 <= 21:
            return total + 10, True  # エースを11として使用可能
        return total, False

    def generate_episode(self):
        """エピソードを生成"""
        player_hand = [self.draw_card(), self.draw_card()]  # プレイヤーの初期手札
        dealer_hand = [self.draw_card(), self.draw_card()]  # ディーラーの初期手札

        player_sum, usable_ace = self.calculate_hand(player_hand)
        dealer_card = dealer_hand[0]

        episode = []
        while True:
            state = (player_sum, dealer_card, usable_ace)
            action = self.policy.get(state, "stick")  # 方策に基づく行動

            episode.append((state, action))

            if action == "stick":  # スティックで終了
                break

            player_hand.append(self.draw_card())
            player_sum, usable_ace = self.calculate_hand(player_hand)

            if player_sum > 21:  # バストで終了
                return episode, -1  # プレイヤーの負け

        # ディーラーのターン
        dealer_sum, _ = self.calculate_hand(dealer_hand)
        while dealer_sum < 17:
            dealer_hand.append(self.draw_card())
            dealer_sum, _ = self.calculate_hand(dealer_hand)

        # 勝敗判定
        if dealer_sum > 21 or player_sum > dealer_sum:
            return episode, 1  # プレイヤーの勝利
        elif player_sum < dealer_sum:
            return episode, -1  # ディーラーの勝利
        else:
            return episode, 0  # 引き分け

    def update_policy(self):
        """モンテカルロESで方策を更新"""
        for _ in range(1000):  # 1000エピソードを実行
            start_state = (random.randint(12, 21), random.randint(1, 10), random.choice([True, False]))
            start_action = random.choice(["hit", "stick"])
            self.policy[start_state] = start_action  # 探索的開始

            episode, reward = self.generate_episode()

            G = reward  # 累積報酬
            visited_pairs = set()
            for state, action in reversed(episode):
                if (state, action) not in visited_pairs:
                    visited_pairs.add((state, action))
                    self.Returns[state][action].append(G)
                    self.Q[state][action] = sum(self.Returns[state][action]) / len(self.Returns[state][action])
                    self.policy[state] = max(self.Q[state], key=self.Q[state].get)  # 方策改善

    def print_policy(self):
        """現在の方策を表示"""
        for state, action in self.policy.items():
            print(f"状態 {state}: 行動 {action}")

# 実行
if __name__ == "__main__":
    mc_es = BlackjackMCES()
    mc_es.initialize_policy()
    print("ブラックジャックでモンテカルロESを実行中...")
    mc_es.update_policy()
    print("\n最適化された方策:")
    mc_es.print_policy()


ブラックジャックでモンテカルロESを実行中...

最適化された方策:
状態 (12, 1, True): 行動 stick
状態 (12, 1, False): 行動 stick
状態 (12, 2, True): 行動 hit
状態 (12, 2, False): 行動 stick
状態 (12, 3, True): 行動 hit
状態 (12, 3, False): 行動 stick
状態 (12, 4, True): 行動 hit
状態 (12, 4, False): 行動 stick
状態 (12, 5, True): 行動 hit
状態 (12, 5, False): 行動 hit
状態 (12, 6, True): 行動 hit
状態 (12, 6, False): 行動 stick
状態 (12, 7, True): 行動 stick
状態 (12, 7, False): 行動 stick
状態 (12, 8, True): 行動 hit
状態 (12, 8, False): 行動 hit
状態 (12, 9, True): 行動 stick
状態 (12, 9, False): 行動 stick
状態 (12, 10, True): 行動 stick
状態 (12, 10, False): 行動 stick
状態 (13, 1, True): 行動 stick
状態 (13, 1, False): 行動 hit
状態 (13, 2, True): 行動 stick
状態 (13, 2, False): 行動 stick
状態 (13, 3, True): 行動 hit
状態 (13, 3, False): 行動 stick
状態 (13, 4, True): 行動 hit
状態 (13, 4, False): 行動 stick
状態 (13, 5, True): 行動 stick
状態 (13, 5, False): 行動 stick
状態 (13, 6, True): 行動 stick
状態 (13, 6, False): 行動 hit
状態 (13, 7, True): 行動 stick
状態 (13, 7, False): 行動 stick
状態 (13, 8, True): 行動 hit
状態 (13, 8, False): 行動 stic